https://captum.ai/tutorials/Image_and_Text_Classification_LIME
This tutorial focuses on showing how to use Captum's implementation of Local Interpretable Model-agnostic Explanations (LIME) to understand neural models. 

In [ ]:
import torch
import torch.nn.functional as F

from captum.attr import visualization as viz
from captum.attr import Lime, LimeBase
from captum._utils.models.linear_model import SkLearnLinearRegression, SkLearnLasso

import os
import json

In [ ]:
from torchvision.models import resnet18
from torchvision.datasets import VOCSegmentation
import torchvision.transforms as T
from captum.attr._core.lime import get_exp_kernel_similarity_function

from PIL import Image
import matplotlib.pyplot as plt

## 1. Load the model and dataset

In [ ]:
resnet = resnet18(pretrained=True)
resnet = resnet.eval()

 ### (train data) ImageNet 1k: mapping of label index and text

In [ ]:
!wget -P $HOME/.torch/models https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json

In [ ]:
labels_path = os.getenv('HOME') + '/.torch/models/imagenet_class_index.json'
with open(labels_path) as json_data:
    idx_to_labels = {idx: label for idx, [_, label] in json.load(json_data).items()}

### (test data) PASCAL VOC 2012 as the 

In [ ]:
voc_ds = VOCSegmentation(
    './VOC',
    year='2012',
    image_set='train',
    download=True, #change to true if not downloaded
    transform=T.Compose([
        T.ToTensor(),
        T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )    
    ]),
    target_transform=T.Lambda(
        lambda p: torch.tensor(p.getdata()).view(1, p.size[1], p.size[0])
    )
)

Let's pick one example to see how the image and corresponding mask look like. Here we choose an image with more than one segments besides background so that we can compare each segment's impact on the classification.

In [ ]:
sample_idx = 439

def show_image(ind): 
    fig, ax = plt.subplots(1, 2, figsize=[6.4 * 2, 4.8])
    for i, (name, source) in enumerate(zip(['Image', 'Mask'], [voc_ds.images, voc_ds.masks])):
        ax[i].imshow(Image.open(source[ind]));
        ax[i].set_title(f"{name} {ind}")
        ax[i].axis('off')

show_image(sample_idx)

## 2. Baseline classification

We can check how well our model works with the above example. The original Resnet only gives the logits of labels, so we will add a softmax layer to normalize them into probabilities.

In [ ]:
img, seg_mask = voc_ds[sample_idx]  # tensors of shape (channel, hight, width)
print('Img shape:',img.shape)
print('Seg shape:',seg_mask.shape)


outputs = resnet(img.unsqueeze(0))
output_probs = F.softmax(outputs, dim=1).squeeze(0)

present the top 5 predicted labels to verify the result.

In [ ]:
def print_result(probs, topk=1):
    probs, label_indices = torch.topk(probs, topk)
    probs = probs.tolist()
    label_indices = label_indices.tolist()
    for prob, idx in zip(probs, label_indices):
        label = idx_to_labels[str(idx)]
        print(f'{label} ({idx}):', round(prob, 4))
        
print_result(output_probs, topk=5)


## 3. Inspect the model prediction with Lime


### 3.1. Generate masks

In case of an image input, the feature mask is a 2D image of the same size, where each pixel in the mask indicates the feature group it belongs to via an integer value. Pixels of the same value define a group.

This means we can readily use VOC's segmentation masks as feature masks for Captum!

In [ ]:
seg_ids = sorted(seg_mask.unique().tolist())
print('Segmentation IDs:', seg_ids)

However, while segmentaion numbers range from 0 to 255, Captum prefers consecutive group IDs for efficiency. Therefore, we will also include extra steps to convert mask IDs.

In [ ]:
# map segment IDs to feature group IDs
feature_mask = seg_mask.clone()
for i, seg_id in enumerate(seg_ids):
    feature_mask[feature_mask == seg_id] = i
    
print('Feature mask:', feature_mask)
print('Feature mask IDs:', feature_mask.unique().tolist())
print('Feature mask shape:',feature_mask.shape)


### 3.2 Lime: train an interpretrable surrogate model

Similarity function. 
Because Lime aims to explain the local behavior of an example, it will reweight the training samples according to their similarity distances. By default, Captum's Lime uses the exponential kernel on top of the consine distance. We will change to euclidean distance instead which is more popular in vision.

In [ ]:
exp_eucl_distance = get_exp_kernel_similarity_function('euclidean', kernel_width=1000)

lr_lime = Lime(
    resnet, 
    interpretable_model=SkLearnLinearRegression(),  # build-in wrapped sklearn Linear Regression
    similarity_func=exp_eucl_distance
)

Next, we will analyze these groups' influence on the most confident prediction television. Every time we call Lime's attribute function, an interpretable model is trained around the given input, so unlike many other Captum's attribution algorithms, it is strongly recommended to only provide a single example as input (tensors with first dimension or batch size = 1).

Setting the `perturbations_per_eval` can batch multiple samples in one forward pass to shorten the process as long as your machine still has capacity

In [ ]:
label_idx = output_probs.argmax().unsqueeze(0)

attrs = lr_lime.attribute(
    img.unsqueeze(0),
    target=label_idx,
    feature_mask=feature_mask.unsqueeze(0),
    n_samples=40,
    perturbations_per_eval=16,
    show_progress=True
).squeeze(0)

print('Label idx (should be tv):',label_idx)
print('Attribution range:', attrs.min().item(), 'to', attrs.max().item())

### 3.3. Visualize the data

In [ ]:
def show_attr(attr_map):
    viz.visualize_image_attr(
        attr_map.permute(1, 2, 0).numpy(),  # adjust shape to height, width, channels 
        method='heat_map',
        sign='all',
        show_colorbar=True
    )
    
show_attr(attrs)

## Now let's do the same, but generating other segments with skimage

In [ ]:
from skimage.segmentation import slic

segmenter=slic(image=img.permute(1, 2, 0) ,n_segments=100)
seg_mask=torch.tensor(segmenter)

seg_ids = sorted(seg_mask.unique().tolist())
print('Segmentation IDs:', seg_ids)
print('Segmentation shape:', seg_mask.shape)

### Visualize the created segments 

In [ ]:
from skimage.segmentation import mark_boundaries
img_wboundaries = mark_boundaries(img.permute(1,2,0).numpy(), segmenter)
plt.imshow(img_wboundaries)

In [ ]:
# map segment IDs to feature group IDs
feature_mask = seg_mask.clone()
for i, seg_id in enumerate(seg_ids):
    feature_mask[feature_mask == seg_id] = i
    
print('Feature mask:', feature_mask)
print('Feature mask IDs:', feature_mask.unique().tolist())
print('Feature mask shape:',feature_mask.shape)

In [ ]:
label_idx = output_probs.argmax().unsqueeze(0)

attrs = lr_lime.attribute(
    img.unsqueeze(0),
    target=label_idx,
    feature_mask=feature_mask.unsqueeze(0),
    n_samples=1000,
    perturbations_per_eval=64,
    show_progress=True
).squeeze(0)

print('Label idx (should be tv):',label_idx)
print('Attribution range:', attrs.min().item(), 'to', attrs.max().item())

In [ ]:

viz.visualize_image_attr(
    attrs.permute(1, 2, 0).numpy(),  # adjust shape to height, width, channels 
    method='heat_map',
    sign='all',
    show_colorbar=True
)

viz.visualize_image_attr(
    attrs.permute(1, 2, 0).numpy(),  # adjust shape to height, width, channels 
    method='blended_heat_map',
    sign='all',
    show_colorbar=True
)